### import libraries

In [1]:
! pip install netCDF4 # python API to work with netcdf (.nc) files

     |████████████████████████████████| 4.1MB 3.9MB/s eta 0:00:0101
     |████████████████████████████████| 317kB 48.9MB/s eta 0:00:01


In [1]:
import netCDF4 # python API to work with netcdf (.nc) files
import os
import datetime 
from osgeo import gdal, ogr, osr
import numpy as np # library to work with matrixes and computations in general
import matplotlib.pyplot as plt # plotting library
from auxiliary_classes import convert_time,convert_time_reverse,kelvin_to_celsius,kelvin_to_celsius_vector,Grid,Image,subImage
import json
import geojson, gdal, subprocess

### auxiliary functions

In [2]:
def print_geojson(tname, tvalue, fname, longitude, latitude, startdoc, position,endloop): #for printing to geojson - start,end,attributes
    fname = fname +".geojson"
    pmode="a"
    if startdoc==1:
        with open(fname, mode="w", encoding='utf-8') as f1: #start of geojson
            tstring = "{\n\"type\": \"FeatureCollection\",\n\"features\": ["
            print(tstring, file=f1)
            f1.close()
    else:
        if position==0: #for printing to geojson - geometry, longitude, latitude
            tstring = "\"type\": \"Feature\",\n\"geometry\": {\n\"type\": \"Point\",\n\"coordinates\": [" + str(longitude) + ","+ str(latitude) + "]\n},\n\"properties\": {"
            fname = fname 
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print(tstring, file=f1)
                f1.close()
        elif position==1:  #start of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print("{", file=f1)
                f1.close()  
        elif position==2: #print attribute (not last)
             with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\","
                print(ttext, file=f1) 
                f1.close() 
        elif position==3: #print last attribute
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\""
                print(ttext, file=f1) 
                f1.close()        
        elif position==4: #end of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:  
                if endloop==0:
                    print("}\n},", file=f1)
                    f1.close()
                else:  #end of geojson
                    print("}\n}\n]\n}", file=f1)
                    f1.close()   
        

In [3]:
def probabilitydate(inputlist, probability, first): #calculate frost date with selected probability from list of frost dates of each year
    listlong = len(inputlist)
    if listlong == 1:
        outputdate = 0
        return outputdate
    elif listlong == 0:
        outputdate = 0
        return outputdate
    else:
        orderlist = orderedlist(inputlist)
        valuelist = daynumberlist(orderlist)
        value = 0
        if first==1:
            value = int(gauss_value(valuelist, probability))
        else: 
            probability=100-probability
            value = int(gauss_value(valuelist, probability))
        outputdate = orderlist[0] + timedelta(days=value)
        return outputdate

In [4]:
def same_year(daylong): #change date to same year (2030) for next calculation
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        sameyear = date(2030, tmonth, tday)
        return sameyear

In [5]:
def gauss_value(inputlist, probability): #value of gaussian probability from values of input list
    mean = np.mean(inputlist)
    sigma = np.std(inputlist)
    values = np.random.normal(mean,sigma,10000)
    value = np.percentile(values,probability)
    return value    

In [6]:
def orderedlist(inputlist): #sort list by date
    listlong = len(inputlist)
    for j in range (0,listlong-1,1):
        for i in range(j+1, listlong, 1):
            firstday = inputlist[j]
            secondday = inputlist[i]
            sfirstday = str(firstday)
            ssecondday = str(secondday)
            fday = int(sfirstday[8:10])
            fmonth = int(sfirstday[5:7])
            sday = int(ssecondday[8:10])
            smonth = int(ssecondday[5:7])
            if fday<10:
                firstval=str(fmonth)+"0"+str(fday)
            else:
                firstval=str(fmonth)+str(fday)
            if sday<10:
                secondval=str(smonth)+"0"+str(sday)
            else:
                secondval=str(smonth)+str(sday)
            firstvalue = int(firstval)
            secondvalue = int(secondval)
            if secondvalue < firstvalue:
                inputlist[j]=secondday
                inputlist[i]=firstday       
    return inputlist

In [7]:
def daynumberlist(orderlist): #from list of ordered dates to list of number of days from first date in ordered list 
    listlong = len(orderlist)
    outputlist=[]
    outputlist.append(0)
    for i in range(1, listlong, 1):
        difference = orderlist[i] - orderlist[0]
        outputlist.append(difference.days)
    return outputlist    

###  Find frost dates: function for one place one year

In [8]:
from datetime import date, timedelta
def findfrostdates(latitude,longitude,year,frostdegree,dayinrow,starthourday,endhourday,fnamefrostdates,im,firstlist, lastlist,nmbfrdayslist,frosthoursday):
      
    numbfrostdays=0 # for calculating number of frost days
        
    #determination of winter and summer:
    wintermonth=1
    summermonth=7
    if latitude<0:
        wintermonth=7
        summermonth=1
    
    # Last spring frost date:
    startmonth=wintermonth
    endmonth=summermonth
    lastfrostday=0
    daysbefore=0
    startdate=1
    enddate=1
    if endmonth == 1:
        endmonth=12
        enddate=31
    sdate = date(year, startmonth, startdate)   # start date for searching last frost date
    edate = date(year, endmonth, enddate)   # end date for searching last frost date
    delta = edate - sdate       # as timedelta
    for i in range(delta.days):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])
        frosthours = 0
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0))
            slice_dictionary={'lon':[longitude,],'lat':[latitude],'time':[int(time)]}
            currenttemp=kelvin_to_celsius_vector(im.slice('tas',slice_dictionary))
            if currenttemp <= frostdegree:
                frosthours+=1
        if frosthours>=frosthoursday:  # frostday?
            numbfrostdays+=1
            lastfrostday=daylong
            if daysbefore>=dayinrow-1:
                lastfrostday=daylong
            daysbefore=+1
        else:
            daysbefore=0
    
    tvarname = "LastD"+ str(year)
    print_geojson(tvarname, lastfrostday, fnamefrostdates, longitude, latitude, 0, 2, 0)
    if lastfrostday!= 0:
        tvalue = same_year(lastfrostday)
        lastlist.append(tvalue)
    
    
                     
    # First autumn frost date:
    startmonth=summermonth
    endmonth=wintermonth
    firstfrostday=0
    daysbefore=0
    cutfrost=0
    enddate=1
    startdate=1
    if endmonth == 1:
        endmonth=12
        enddate=31
        
        
    sdate = date(year, startmonth, startdate)   # start date of searching
    edate = date(year, endmonth, enddate)   # end date of searching
    delta = edate - sdate       # as timedelta
    for i in range(delta.days+1):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])
        frosthours = 0 
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0))
            slice_dictionary={'lon':[longitude],'lat':[latitude],'time':[int(time)]}
            currenttemp=kelvin_to_celsius_vector(im.slice('tas',slice_dictionary))
            if currenttemp <= frostdegree:
                frosthours+=1
        if frosthours >= frosthoursday:  # frostday?
            numbfrostdays+=1
            if daysbefore>=(dayinrow-1) and cutfrost==0:
                firstfrostday=daylong
                cutfrost=1
            daysbefore=+1
        else:
            daysbefore=0
       
    tvarname = "FirstD"+str(year)
    print_geojson(tvarname, firstfrostday, fnamefrostdates, longitude, latitude, 0, 2, 0)
    if firstfrostday!= 0:
        tvalue = same_year(firstfrostday)
        firstlist.append(tvalue)
    # Frostfreeperiod
    frostfreeperiod=0
    if firstfrostday!=0 and lastfrostday!=0:
        if latitude>0:
            frostfreeperiod =  firstfrostday-lastfrostday
            frostfreeperiod = frostfreeperiod.days
        else:
            firstyeardate = date(year, 1, 1)   # start date of year
            lastyeardate = date(year, 12, 31)   # end date of year
            frostfreeperiod =  (firstfrostday-firstyeardate)+(lastyeardate-lastfrostday)
    tvarname = "Period"+ str(year)
    print_geojson(tvarname, frostfreeperiod, fnamefrostdates, longitude, latitude, 0, 2, 0)
    
    tvarname = "FrDays"+ str(year) 
    print_geojson(tvarname, numbfrostdays, fnamefrostdates, longitude, latitude, 0, 2, 0)
    nmbfrdayslist.append(numbfrostdays)
    
                  

### Find frost dates: iteration by selected years

In [9]:
def frostdatesyearly(latorder,lonorder,startyear,endyear,frostdegree,dayinrow,starthourday,endhourday,fnamefrostdates,endloop,datafolder,probability,frosthoursday):
    print_geojson("", "", fnamefrostdates, 0, 0, 0, 1,0)
    firstlist = []
    lastlist = []
    nmbfrdayslist = []
    
    for year in range(startyear, endyear+1, 1):
        source = datafolder + '/' + str(year) + '.nc' 
        im=Image(netCDF4.Dataset(source,'r'))   
        longlist = im.get_data().variables['lon'][:]
        latlist= im.get_data().variables['lat'][:]
        longitude = longlist [lonorder]   
        latitude = latlist[latorder]
        if year == startyear:
            print_geojson("", "", fnamefrostdates, longitude, latitude, 0, 0,0)
        findfrostdates(latitude,longitude,year,frostdegree,dayinrow,starthourday,endhourday,fnamefrostdates,im,firstlist,lastlist,nmbfrdayslist,frosthoursday)
    
    firstprobday = probabilitydate(firstlist, probability, 1)
    lastprobday = probabilitydate(lastlist, probability, 0)
    namefirstprob = "FirstD" + str(probability) 
    namelastprob = "LastD" + str(probability) 
    print_geojson(namelastprob, lastprobday, fnamefrostdates, 0, 0, 0, 2, 0)
    print_geojson(namefirstprob, firstprobday, fnamefrostdates, 0, 0, 0, 2, 0)
    
    nameperiodprob = "Period" + str(probability) 
    frostfreeperiod=0
    if firstprobday!=0 and lastprobday!=0:
        if latitude>0:
            frostfreeperiod =  firstprobday-lastprobday
            frostfreeperiod = frostfreeperiod.days
        else:
            firstyeardate = date(2030, 1, 1)   # start date of year
            lastyeardate = date(2030, 12, 31)   # end date of year
            frostfreeperiod =  (firstprobday-firstyeardate)+(lastyeardate-lastprobday) 
    
    print_geojson(nameperiodprob, frostfreeperiod, fnamefrostdates, 0, 0, 0, 2, 0)
    tmeannmb = np.mean(nmbfrdayslist)
    meannmb = int(np.round(tmeannmb, decimals=0, out=None))
    print_geojson("AvgFrDays", meannmb, fnamefrostdates, 0, 0, 0, 3, 0)
    print_geojson("", "", fnamefrostdates, 0, 0, 0, 4,endloop)
          

### Find frost dates: iteration by selected latitudes, longitudes

In [10]:
def frostdatesplaces(startlat, startlon, endlat, endlon, startyear,endyear,frostdegree,dayinrow,starthourday,endhourday,exportfolder,datafolder,fnamefrostdates1,probability,frosthoursday,alllatlonfile):
        fnamefrostdates = exportfolder + "/" +fnamefrostdates1
        print_geojson("", "", fnamefrostdates, 0, 0, 1, 0,0)
        endloop=0
        
        if alllatlonfile==1:  # if it is calculated for all latitudes and longitudes in input file
            source = datafolder + '/' + str(startyear) + '.nc' 
            im=Image(netCDF4.Dataset(source,'r')) 
            arraylon = im.get_data().variables['lon'][0::]
            arraylat = im.get_data().variables['lat'][0::]
            startlat=0
            startlon=0
            endlon= len(arraylon)-1
            endlat= len(arraylat)-1
             
        
        for latorder in range(startlat, endlat+1, 1):
            for lonorder in range(startlon, endlon+1, 1):
                if latorder==endlat and lonorder==endlon:
                    endloop=1
                frostdatesyearly(latorder,lonorder,startyear,endyear,frostdegree,dayinrow,starthourday,endhourday,fnamefrostdates,endloop,datafolder,probability,frosthoursday)
        
       
        

## <font color=red>Find frost dates: input parameters and launch</font> 

In [13]:
#Frost date definition:
frostdegree=0 # destructive minimum temperature for the plant # we can find last/first day with 0/-1/-2/.... degrees of Celsius # integer or double
dayinrow=1 #how many days in a row we consider as a frost date, how many repetitions are destructive to a plant #we can find last/first frost date or for example last/first two frost days of each year # integer
frosthoursday=1 #the number of frost hours per day from which the day is considered as the frost day #some plants may not be damaged by only one frost hour a day, but more frost hours may damaged them

#Time and probability definition:
startyear=1982 #start year (integer) 
endyear=2019 #end year (integer)  
probability=50 # probability (percent) of frost date (integer 10-90)

#Optimalization:
starthourday=0 # integer 0-23, we can select only sunrise hours
endhourday=23 # integer 0-23, we can select only sunrise hours

#Files/Folders name:
datafolder = "data" #folder with data files (named by year) for each year #string
fnamefrostdates="frostdates" #name of created files #string
exportfolder = "export" #for all files (if each file its folder -> insert name of folder to name of file) #export folder must be created #string

#Area definition:
alllatlonfile=0 #calculate all latitudes and longitudes in input file (1=yes, 0=no)
# if alllatlonfile!=0 then:
startlat=0 # start number of list of latitudes from used netCDF4 file 
startlon=0 # start number of list of longitudes from used netCDF4 file 
endlat=1 # end number of list of latitudes from used netCDF4 file 
endlon=1 # end number of list of longitudes from used netCDF4 file 
 

    
frostdatesplaces(startlat, startlon, endlat, endlon, startyear,endyear,frostdegree,dayinrow,starthourday,endhourday,exportfolder,datafolder,fnamefrostdates,probability,frosthoursday,alllatlonfile)






<font color=red> Output: in export folder is created geojson with points - each point has got attributes: first/last frost date, frost-free period with corresponding probability, frost dates, period for each year and also number of frost days in each year or averange of number of frost days </font> 

## From geojson to shp

In [31]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/frostdates.shp', 'export/frostdates.geojson']
subprocess.Popen(args)